# Acrobot

## A2C Agent 

In [ ]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "Acrobot-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "Acrobot-v1":
    gamma = 0.95
    lr = 7e-4

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

In [ ]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (KL) 

In [11]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = "Acrobot-v1"
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent

# When the learning rate is large, policy neural network can overflow and lead to NaNs. 
# A possible fix is to reduce lr or increase beta to lower the learning rate.

if kl_env == "Acrobot-v1":
    gamma = 0.95
    lr = 1e-2
    beta = 0.8

agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration
    beta += np.random.random()*0.1
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward  <= -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2
        
    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= -490):
        agent = DRTRPOAgent(env, gamma, lr)

    policy_loss = agent.compute_policy_loss_kl(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -500.0
Episode 5: -500.0
Episode 6: -500.0
Episode 7: -214.33333333333334
Episode 8: -170.0
Episode 9: -193.0
Episode 10: -154.33333333333334
Episode 11: -166.33333333333334
Episode 12: -133.66666666666666
Episode 13: -155.33333333333334
Episode 14: -144.66666666666666
Episode 15: -132.0
Episode 16: -233.0
Episode 17: -200.33333333333334
Episode 18: -138.33333333333334
Episode 19: -157.66666666666666
Episode 20: -164.33333333333334
Episode 21: -150.33333333333334
Episode 22: -154.0
Episode 23: -159.33333333333334
Episode 24: -206.0
Episode 25: -139.33333333333334
Episode 26: -163.33333333333334
Episode 27: -154.33333333333334
Episode 28: -278.3333333333333
Episode 29: -154.0
Episode 30: -136.66666666666666
Episode 31: -145.66666666666666
Episode 32: -123.0
Episode 33: -176.66666666666666
Episode 34: -178.0
Episode 35: -184.0
Episode 36: -138.33333333333334
Episode 37: -147.33333333333334
Episode 38: -143

Episode 290: -204.0
Episode 291: -152.33333333333334
Episode 292: -174.0
Episode 293: -137.33333333333334
Episode 294: -166.33333333333334
Episode 295: -158.66666666666666
Episode 296: -130.0
Episode 297: -106.33333333333333
Episode 298: -128.0
Episode 299: -156.33333333333334


In [12]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Wasserstein)

In [14]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "Acrobot-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "Acrobot-v1":
    gamma = 0.95
    lr = 1e-2
    
agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
    
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration 
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward < -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2

    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= -490):
        agent = DRTRPOAgent(env, gamma, lr)
        
    total_adv_diff += max(abs(state_adv[1] - state_adv[0]), abs(state_adv[2] - state_adv[0]), abs(state_adv[2] - state_adv[1]))
    beta = total_adv_diff/episode
    beta_l = 0.6
    beta_s = 0.1
    policy_loss = agent.compute_policy_loss_wass(first_state, state_adv, beta_s, beta_l)

    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: -498.3333333333333
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -221.33333333333334
Episode 5: -159.0
Episode 6: -160.0
Episode 7: -175.0
Episode 8: -132.66666666666666
Episode 9: -144.33333333333334
Episode 10: -162.66666666666666
Episode 11: -130.66666666666666
Episode 12: -157.66666666666666
Episode 13: -130.66666666666666
Episode 14: -111.66666666666667
Episode 15: -133.66666666666666
Episode 16: -117.33333333333333
Episode 17: -121.66666666666667
Episode 18: -134.0
Episode 19: -134.33333333333334
Episode 20: -138.33333333333334
Episode 21: -100.33333333333333
Episode 22: -150.66666666666666
Episode 23: -129.0
Episode 24: -126.33333333333333
Episode 25: -114.0
Episode 26: -118.0
Episode 27: -127.33333333333333
Episode 28: -123.0
Episode 29: -117.66666666666667
Episode 30: -120.0
Episode 31: -118.33333333333333
Episode 32: -132.0
Episode 33: -135.0
Episode 34: -140.0
Episode 35: -148.0
Episode 36: -114.33333333333333
Episode 37: -127.33333333333333
Epi

Episode 294: -145.0
Episode 295: -100.66666666666667
Episode 296: -117.66666666666667
Episode 297: -100.33333333333333
Episode 298: -106.0
Episode 299: -105.33333333333333


In [15]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')